In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from  keras.wrappers.scikit_learn  import  KerasClassifier
from  sklearn.model_selection  import  GridSearchCV

import numpy as np
import pandas as pd

In [2]:
#part A)
df = pd.read_csv("https://raw.githubusercontent.com/FinalformSora/StockML/main/stocks%20v1%20-%20master%20sheet.csv")
df[0::10]
df.size

35680

In [3]:
#Data that  has been parsed and cleaned up
#Had to create a moving average for 50 day and 100 day. It requres calculation of previous days and finding the average
#increase is based on CLose-Open= if it equals positive its a true on increase
#AMD news is based on press release released by amd on their site. I had to manual add this in cause no site has this information parsed
#quarterly earning is also gather by searching the web for this release date
df

,Date,Stock name,Open,High,Low,Close*,Adj Close**,Volume,volatility,Volitity (1 or 0),mv50,Higher then MVA,MV100,Higher then MVA.1,EMA 50,Higher then EMVA
0,4/8/2021,S&P500,"4,089.95","4,098.19","4,082.54","4,097.17","4,097.17","3,901,910,000",No,0,3906.5626,Yes,3893.7234,Yes,3909.064049,No
1,4/7/2021,S&P500,"4,074.29","4,083.13","4,068.31","4,079.95","4,079.95","4,112,640,000",No,0,3901.6116,Yes,3888.9029,Yes,3901.386255,No
2,4/6/2021,S&P500,"4,075.57","4,086.23","4,068.14","4,073.94","4,073.94","4,027,880,000",No,0,3897.1198,Yes,3883.4554,Yes,3894.097938,No
3,4/5/2021,S&P500,"4,034.44","4,083.42","4,034.44","4,077.91","4,077.91","3,999,760,000",No,0,3892.4704,Yes,3878.3137,Yes,3886.757446,No
4,4/1/2021,S&P500,"3,992.78","4,020.63","3,992.78","4,019.87","4,019.87","4,151,240,000",No,0,3887.9736,Yes,3873.0710,Yes,3878.955301,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,3/31/2021,CokeCola,294.35,297.74,288.59,288.78,288.78,34100,No,0,276.9804,Yes,272.1605,Yes,280.077844,No
2226,4/1/2021,CokeCola,287.78,289.94,286.95,289.17,289.17,22800,No,0,277.6396,Yes,272.7301,Yes,280.434399,No
2227,4/5/2021,CokeCola,288.15,298.87,288,295.19,295.19,30000,No,0,278.3934,Yes,273.2670,Yes,281.013050,No
2228,4/6/2021,CokeCola,297.62,301.77,296.14,296.36,296.36,21400,No,0,279.0738,Yes,273.8556,Yes,281.614891,No


In [4]:
#creating the Colunms for important data to use

In [5]:
#feature_cols = ['Volume','50 moving day average (1 or 0)','100 day moving average (1 or 0)','AMD News (1 or 0)','Quarterly earnings (1 or 0)']
feature_cols = ['Stock name','Open','High','Low','Volitity (1 or 0)','Volume','mv50','MV100','EMA 50']
X = df[feature_cols]
X.head()



,Stock name,Open,High,Low,Volitity (1 or 0),Volume,mv50,MV100,EMA 50
0,S&P500,"4,089.95","4,098.19","4,082.54",0,"3,901,910,000",3906.5626,3893.7234,3909.064049
1,S&P500,"4,074.29","4,083.13","4,068.31",0,"4,112,640,000",3901.6116,3888.9029,3901.386255
2,S&P500,"4,075.57","4,086.23","4,068.14",0,"4,027,880,000",3897.1198,3883.4554,3894.097938
3,S&P500,"4,034.44","4,083.42","4,034.44",0,"3,999,760,000",3892.4704,3878.3137,3886.757446
4,S&P500,"3,992.78","4,020.63","3,992.78",0,"4,151,240,000",3887.9736,3873.0710,3878.955301


In [6]:
#created the label based on if the stock increased or did not increase

In [7]:
y = df['Close*']
y[::25]

0       4,097.17
25      3,819.72
50      3,849.62
75      3,722.48
100     3,572.66
          ...   
2125      232.21
2150         261
2175      255.56
2200      262.75
2225      288.78
Name: Close*, Length: 90, dtype: object

In [8]:
#calling all the classifers
my_logreg = LogisticRegression()
my_decisiontree = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=4)
my_RandomForest = RandomForestClassifier()
my_linreg = LinearRegression()
my_AdaBoost = AdaBoostClassifier(n_estimators = 29,random_state=2)
my_XGBoost = XGBClassifier(n_estimators = 29,random_state=2)


In [9]:
#seting up train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [11]:
from keras.utils import np_utils

X_train = np_utils.to_categorical(X_train, 4)
X_test = np_utils.to_categorical(X_test, 4)

ValueError: invalid literal for int() with base 10: 'Gamestop'

In [10]:
# training sets

#knn.fit(X_train, y_train)
my_logreg.fit(X_train, y_train)
my_RandomForest.fit(X_train, y_train)
my_decisiontree.fit(X_train, y_train)
my_linreg.fit(X_train, y_train)
my_AdaBoost.fit(X_train,y_train)
my_XGBoost.fit(X_train,y_train)


C:\Users\jesus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'CokeCola'

In [ ]:
#Training
y_predict_lor = my_logreg.predict(X_test)
y_predict = knn.predict(X_test)
y_predict_dt = my_decisiontree.predict(X_test)
y_predict_rt = my_RandomForest.predict(X_test)
y_predict_LR = my_linreg.predict(X_test)
y_predict_AB = my_AdaBoost.predict(X_test)
y_predict_XB = my_XGBoost.predict(X_test)


In [ ]:
#accuracy of the perdiction
from sklearn.metrics import accuracy_score

score_lor = accuracy_score(y_test, y_predict_lor)
score_dt = accuracy_score(y_test, y_predict_dt)
accuracy = accuracy_score(y_test, y_predict)
score_rt = accuracy_score(y_test,y_predict_rt)
score_AB = accuracy_score(y_test, y_predict_AB)
score_XB = accuracy_score(y_test, y_predict_XB)

y_predict_LR = my_linreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_predict_LR)
rmse = np.sqrt(mse)
print("Linear Regression",rmse)
print('LogisticRegression: ',score_lor)
print("Decision Tree: ",score_dt)
print("Random Forest: ",score_rt)
print("knn: ",accuracy)
print("AdaBoost: ",score_AB)
print("XGBoost: ",score_XB)


In [ ]:
#Cross validation of all Classifers

In [ ]:
accuracy_list = cross_val_score(my_logreg, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()
print("Logistic Regression: ",accuracy_cv)

In [ ]:
accuracy_list = cross_val_score(my_RandomForest, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()

print("Random Forest: ",accuracy_cv)

In [ ]:
accuracy_list = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()

print("KNN: ",accuracy_cv)


In [ ]:
accuracy_list = cross_val_score(my_decisiontree, X, y, cv=10, scoring='accuracy')
accuracy_cv = accuracy_list.mean()

print("decisiontree: ",accuracy_cv)

In [ ]:
mse_list = cross_val_score(my_linreg, X, y, cv=10, scoring='neg_mean_squared_error')
mse_list_positive = -mse_list
rmse_list = np.sqrt(mse_list_positive)
print("Linear Regression: ",rmse_list.mean())


In [ ]:
accuracy_list = cross_val_score(my_AdaBoost, X, y, cv=10, scoring='accuracy')
accuracy_ab = accuracy_list.mean()

print("Adaboost : ",accuracy_ab)

In [ ]:
accuracy_list = cross_val_score(my_XGBoost, X, y, cv=10, scoring='accuracy')
accuracy_XG = accuracy_list.mean()

print("XGBoost: ",accuracy_XG)

In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing

my_ANN = MLPClassifier(hidden_layer_sizes=(30,), activation= 'logistic', 
                       solver='adam', alpha=1, random_state=1, 
                       learning_rate_init = 0.02, verbose=True)

from sklearn.model_selection import cross_val_score
accuracy_list = cross_val_score(my_ANN, preprocessing_features, y, cv=10, scoring='accuracy')

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

seed = 2
np.random.seed(seed)

model = KerasClassifier(build_fn = model_creator, verbose=2)


batch_size = [30 , 50 , 100 ]
epochs = [10 , 15 , 20]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(model, param_grid, cv=10, scoring='neg_log_loss')


In [ ]:
my_grid = grid.fit(X_train, y_train)

print("best: %f with %s" % (my_grid.best_score_, my_grid.best_params_))